# This is only a incomplete API framework to send requests to Maayan Lab databases. We are not currently using this.

In [ ]:
import requests
import os
import json  # Import the correct JSON module

# API endpoint
api_url = "https://maayanlab.cloud/Harmonizome/api/1.0/dataset/LINCS+L1000+CMAP+Signatures+of+Differentially+Expressed+Genes+for+Small+Molecules"

# Fetch the API response
response = requests.get(api_url)
if response.status_code != 200:
    print(f"Failed to fetch API data. HTTP Status Code: {response.status_code}")
    exit()

# Parse the JSON response
try:
    data = response.json()
except ValueError:
    print("Failed to parse JSON response.")
    exit()

# Extract gene sets
gene_sets = data.get("geneSets", [])
if not gene_sets:
    print("No gene sets found in the API response.")
    exit()

# Create a folder for the dataset
dataset_name = data.get("name", "dataset").replace(" ", "_")
if not os.path.exists(dataset_name):
    os.mkdir(dataset_name)

# Download each gene set
for gene_set in gene_sets:
    name = gene_set["name"]
    href = gene_set["href"]
    # Replace spaces with '+' in the href to construct the correct URL
    corrected_href = href.replace(" ", "+")
    file_url = f"https://maayanlab.cloud/Harmonizome/{corrected_href}"

    print(f"Fetching data for: {name}")
    print(f"Constructed URL: {file_url}")  # Debugging output

    file_response = requests.get(file_url)
    if file_response.status_code == 200:
        try:
            # Parse the gene set data
            gene_set_data = file_response.json()
            # Save the data as a JSON file
            file_path = os.path.join(dataset_name, name.replace("/", "_") + ".json")
            with open(file_path, "w") as f:
                json.dump(gene_set_data, f, indent=4)  # Use the correct json.dump method
            print(f"Saved: {file_path}")
        except ValueError:
            print(f"Failed to parse JSON for {name}.")
    else:
        print(f"Failed to fetch data for {name}. HTTP Status Code: {file_response.status_code}")